In [1]:
(require '[clojupyter.misc.helper :as helper])
(helper/add-dependencies '[midje "1.9.1"])
(use 'clojure.pprint)
(use 'clojure.repl)
(use 'midje.repl)

Run `(doc midje)` for Midje usage.
Run `(doc midje-repl)` for descriptions of Midje repl functions.


# Functional Programming (cont.)

## Cool Things to Do with Pure Functions

### Composition - `comp`

In [2]:
(facts "about `comp`"
       (fact ((comp inc *) 2 3) => (inc (* 2 3))))

true

In [3]:
(def character
    {:name "Smooches McCutes"
     :attributes {:intelligence 10
                  :strength 4
                  :dexterity 5}})

(def c-int (comp :intelligence :attributes))
(def c-str (comp :strength :attributes))
(def c-dex (comp :dexterity :attributes))

(facts "about character"
       (fact (c-int character) => 10)
       (fact (c-str character) => 4)
       (fact (c-dex character) => 5))

(def m-int (fn [c] (:intelligence (:attributes c))))
(def m-str (fn [c] (:strength (:attributes c))))
(def m-dex (fn [c] (:dexterity (:attributes c))))

(facts "about character"
       (fact (m-int character) => 10)
       (fact (m-str character) => 4)
       (fact (m-dex character) => 5))

true

In [4]:
(defn spell-slots
    "calculates the number of spell slots your character has based on her intelligence attribute"
    [char]
    (int (inc (/ (c-int char) 2))))

(fact (spell-slots character) => 6)

true

In [5]:
; to use a two argument function w/ comp, wrap it in an anonymous function
(def spell-slots-comp (comp int inc #(/ % 2) c-int))
(fact (spell-slots-comp character) => 6)

true

In [6]:
(defn two-comp
    [f g]
    (fn [& args]
        (f (apply g args))))

(fact ((two-comp inc *) 2 3) => 7)

true

In [7]:
(defn chain
    "simple function for composing two functions"
    [f g]
    (fn [& args]
        (f (apply g args))))

(defn my-comp
    "compose a collection of functions together"
    [& fs]
    (reduce chain identity fs))

(facts
    (fact ((comp rest reverse) [1 2 3 4]) => [3 2 1])
    (fact ((my-comp rest reverse) [1 2 3 4]) => [3 2 1])
    (fact ((my-comp (partial + 3) second) [1 2 3 4]) => 5)
    (fact ((my-comp zero? #(mod % 8) +) 3 5 7 9) => true)
    (fact ((my-comp #(.toUpperCase %) #(apply str %) take) 5 "hello world") => "HELLO"))

true

### Memoization - `memoize`

In [8]:
(defn sleepy-identity
    "Returns the given value after 1 second"
    [x]
    (Thread/sleep 1000)
    x)

(time (sleepy-identity "Mr. Fantastico"))

"Elapsed time: 1003.345562 msecs"


"Mr. Fantastico"

In [9]:
(time (sleepy-identity "Mr. Fantastico"))

"Elapsed time: 1004.080673 msecs"


"Mr. Fantastico"

In [10]:
(def memo-sleepy-identity (memoize sleepy-identity))
(time (memo-sleepy-identity "Mr. Fantastico"))

"Elapsed time: 1004.643172 msecs"


"Mr. Fantastico"

In [11]:
(time (memo-sleepy-identity "Mr. Fantastico"))

"Elapsed time: 0.06724 msecs"


"Mr. Fantastico"

## Peg Thing

1. `connect-right` - if a number `n` is a triangular number, then it is on the right boundary. if `pos` or `neighbor` are triangular numbers, then you cannot make a move to the right.
2. `connect-down-left` &amp; `connect-down-right` - down-diagonal moves can only every violate the bottom boundary and this is checked in the call to `connect`.

Reducing can be used over a collection of either data _or functions_

In [12]:
(require '[clojure.string :as s])
(defn clean-reduce
    [text]
    (reduce (fn [string string-fn] (string-fn string))
            text
            [s/trim #(s/replace % #"lol" "LOL")]))

(defn clean-comp
    [text]
    ((comp s/trim #(s/replace % #"lol" "LOL")) text))

(facts "about cleaning text"
       (fact (clean-reduce "My boa constrictor is so sassy lol!  ") => "My boa constrictor is so sassy LOL!")
       (fact (clean-comp "My boa constrictor is so sassy lol!  ") => "My boa constrictor is so sassy LOL!"))

true

3. `new-board` - because we only try connecting right, down-left, &amp; down-right this function does not create duplicate connections
4. `remove-peg`, `place-peg`, &amp; `move-peg` do not do bounds checks
5. `pegged?` might return `nil`

In [13]:
; use `do-seq` to have a side-effect
(doseq [i (range 1 (inc 10))]
        (pprint i))

1
2
3
4
5
6
7
8
9
10
